---
<h1><center> 2. Capstone Scrapers </center></h1>


--- 

This notebook contains the code used to pull information from the nba.com website. It's very rough and uncommented as I plan to post each individual scraper code on my github. While pulling data from NBA.com I realized I was missing quite a bit of data relating to the location of the defenders. My first scraper enabled me to get individual shot locations as NBA.com only provided defender distances on an aggregated level. Luckily I was able to detailed shot information on github from the 2015 season before the NBA changed the amount of data they provide to the public. The three main scrapers in this notebook allowed me to get Shot Locations, offensive player data and defensive player data.

In [2]:
import pandas as pd
import numpy as np
import requests
import json
import time
import warnings
warnings.filterwarnings('ignore')
import csv

In [4]:
all_players_list = pd.DataFrame.from_csv('data/playerlist.csv')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


In [5]:
all_players_list.head()

,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,FROM_YEAR,GAMES_PLAYED_FLAG,PERSON_ID,PLAYERCODE,ROSTERSTATUS,TEAM_ABBREVIATION,TEAM_CITY,TEAM_CODE,TEAM_ID,TEAM_NAME,TO_YEAR
0,Alaa Abdelnaby,"Abdelnaby, Alaa",1990,Y,76001,HISTADD_alaa_abdelnaby,0,NaN,NaN,NaN,0,NaN,1994
1,Zaid Abdul-Aziz,"Abdul-Aziz, Zaid",1968,Y,76002,HISTADD_zaid_abdul-aziz,0,NaN,NaN,NaN,0,NaN,1977
2,Kareem Abdul-Jabbar,"Abdul-Jabbar, Kareem",1969,Y,76003,HISTADD_kareem_abdul-jabbar,0,NaN,NaN,NaN,0,NaN,1988
3,Mahmoud Abdul-Rauf,"Abdul-Rauf, Mahmoud",1990,Y,51,mahmoud_abdul-rauf,0,NaN,NaN,NaN,0,NaN,2000
4,Tariq Abdul-Wahad,"Abdul-Wahad, Tariq",1997,Y,1505,tariq_abdul-wahad,0,NaN,NaN,NaN,0,NaN,2003


In [78]:
all_players_list.columns

Index(['DISPLAY_FIRST_LAST', 'DISPLAY_LAST_COMMA_FIRST', 'FROM_YEAR',
       'GAMES_PLAYED_FLAG', 'PERSON_ID', 'PLAYERCODE', 'ROSTERSTATUS',
       'TEAM_ABBREVIATION', 'TEAM_CITY', 'TEAM_CODE', 'TEAM_ID', 'TEAM_NAME',
       'TO_YEAR'],
      dtype='object')

In [6]:
all_players_list.shape

(4283, 13)

In [7]:
player_list = all_players_list[['PERSON_ID','DISPLAY_FIRST_LAST', 'FROM_YEAR', 'TO_YEAR']]

In [8]:
player_list.head()

,PERSON_ID,DISPLAY_FIRST_LAST,FROM_YEAR,TO_YEAR
0,76001,Alaa Abdelnaby,1990,1994
1,76002,Zaid Abdul-Aziz,1968,1977
2,76003,Kareem Abdul-Jabbar,1969,1988
3,51,Mahmoud Abdul-Rauf,1990,2000
4,1505,Tariq Abdul-Wahad,1997,2003


In [20]:
player_list['TO_YEAR'].dtype

dtype('int64')

In [28]:
player_list = player_list[player_list.FROM_YEAR > 1990]

In [29]:
player_list.shape

(1963, 4)

In [30]:
player_list = player_list[player_list.TO_YEAR > 2014]

In [32]:
player_list.shape

(717, 4)

In [37]:
player_list.head()

,PERSON_ID,DISPLAY_FIRST_LAST,FROM_YEAR,TO_YEAR
9,203518,Alex Abrines,2016,2017
14,203112,Quincy Acy,2012,2017
18,203919,Jordan Adams,2014,2015
20,203500,Steven Adams,2013,2017
22,1628389,Bam Adebayo,2017,2017


In [38]:
player_list = player_list[['PERSON_ID', 'DISPLAY_FIRST_LAST']]

In [39]:
player_list.head()

,PERSON_ID,DISPLAY_FIRST_LAST
9,203518,Alex Abrines
14,203112,Quincy Acy
18,203919,Jordan Adams
20,203500,Steven Adams
22,1628389,Bam Adebayo


In [42]:
player_ids = [player_list['PERSON_ID']]

In [54]:
type(player_ids[0].tolist())
player_ids = player_ids[0].tolist()

In [55]:
player_ids

[203518,
 203112,
 203919,
 203500,
 1628389,
 201167,
 201582,
 202332,
 200746,
 1626146,
 1628386,
 1628443,
 202730,
 2754,
 202329,
 200811,
 2365,
 101187,
 202341,
 1626147,
 203937,
 201583,
 1628387,
 203507,
 203648,
 2546,
 201202,
 1628384,
 203951,
 1627853,
 2772,
 201589,
 1628503,
 201600,
 201571,
 201965,
 202337,
 1628407,
 203946,
 1627758,
 1627735,
 1628366,
 2571,
 200826,
 200745,
 203084,
 2440,
 203115,
 101138,
 201587,
 201573,
 203382,
 203145,
 203078,
 1627736,
 201563,
 201158,
 1628395,
 1627761,
 1627733,
 203461,
 1627791,
 202722,
 201976,
 203920,
 1628444,
 202687,
 202357,
 204028,
 201971,
 2581,
 1628469,
 202339,
 203505,
 203992,
 202711,
 101106,
 1627762,
 2588,
 1626164,
 202344,
 2547,
 1628449,
 202340,
 1628396,
 1882,
 201147,
 1627763,
 201166,
 1628415,
 202705,
 1626148,
 201628,
 1627759,
 203485,
 203900,
 1628425,
 1627852,
 977,
 1628418,
 201978,
 203493,
 203504,
 202692,
 2406,
 202710,
 2446,
 202779,
 203998,
 101181,
 20348

In [85]:
csvfile = 'data/player_ids'

with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in player_ids:
        writer.writerow([val])   

In [56]:
#MULTIPLE YEARS
sc_url_1 = 'https://stats.nba.com/stats/shotchartdetail?AheadBehind=&CFID=33&CFPARAMS='
sc_url_2 = '&ClutchTime=&Conference=&ContextFilter=&ContextMeasure=FGA&DateFrom=&DateTo=&Division=&EndPeriod=10&EndRange=28800&GROUP_ID=&GameEventID=&GameID=&GameSegment=&GroupID=&GroupMode=&GroupQuantity=5&LastNGames=0&LeagueID=00&Location=&Month=0&OnOff=&OpponentTeamID=0&Outcome=&PORound=0&Period=0&PlayerID='
sc_url_3 = '&PlayerID1=&PlayerID2=&PlayerID3=&PlayerID4=&PlayerID5=&PlayerPosition=&PointDiff=&Position=&RangeType=0&RookieYear=&Season='
sc_url_4 = '&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StartPeriod=1&StartRange=0&StarterBench=&TeamID=0&VsConference=&VsDivision=&VsPlayerID1=&VsPlayerID2=&VsPlayerID3=&VsPlayerID4=&VsPlayerID5=&VsTeamID='

headers = requests.utils.default_headers()
headers.update({
    "user-agent": 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',
    })

#year in yyyy-yy format (i.e. '2017-18')
def get_all_players_shot_data(player_ids, year):
    all_shots = []
    c=0
    start=time.time()
    for player_id in player_ids:
        full_url = sc_url_1 + str(year) + sc_url_2 + str(player_id) + sc_url_3 + str(year) + sc_url_4
        shots = requests.get(full_url, headers=headers).json()
        all_shots.append(shots)
        time.sleep(1)
        c+=1
        if c%50==0:
            print('Runtime: {} seconds. {} players completed'.format(time.time()-start, c))
    return all_shots

def convert_dict_to_df(all_shot_data):
    start=time.time()

    league_avgs = all_shot_data[0]['resultSets'][1]['rowSet']
    league_avg_columns = all_shot_data[0]['resultSets'][1]['headers']
    league_avgs_df = pd.DataFrame.from_records(league_avgs, columns=league_avg_columns)

    columns = all_shot_data[0]['resultSets'][0]['headers']

    df_list=[]

    for player in all_shot_data:
        data = player['resultSets'][0]['rowSet']
        player_df = pd.DataFrame.from_records(data, columns=columns)
        df_list.append(player_df)

    df = pd.concat(df_list, ignore_index=True)
    print('Total Runtime: {} seconds.'.format(time.time()-start))

    return df, league_avgs_df

all_shots_1415 = get_all_players_shot_data(player_ids, '2014-15')
shots_1415_df, lg_avgs_1415 = convert_dict_to_df(all_shots_1415)

shots_1415_df.to_csv('data/shots_1415.csv')

Runtime: 57.738208055496216 seconds. 50 players completed
Runtime: 115.07853078842163 seconds. 100 players completed
Runtime: 202.57730388641357 seconds. 150 players completed
Runtime: 288.59418773651123 seconds. 200 players completed
Runtime: 379.93022084236145 seconds. 250 players completed
Runtime: 465.02724289894104 seconds. 300 players completed
Runtime: 549.7054228782654 seconds. 350 players completed
Runtime: 635.9309258460999 seconds. 400 players completed
Runtime: 720.4094579219818 seconds. 450 players completed
Runtime: 806.2132518291473 seconds. 500 players completed
Runtime: 890.3916029930115 seconds. 550 players completed
Runtime: 976.2018220424652 seconds. 600 players completed
Runtime: 1063.6513738632202 seconds. 650 players completed
Runtime: 1149.0490329265594 seconds. 700 players completed
Total Runtime: 1.7570738792419434 seconds.


In [57]:
shots_1415 = pd.read_csv('data/shots_1415.csv')

In [58]:
shots_1415.head()

,Unnamed: 0,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,0,Shot Chart Detail,21400010,107,203112,Quincy Acy,1610612752,New York Knicks,1,1,...,Right Side(R),16-24 ft.,18,185,15,1,1,20141029,NYK,CHI
1,1,Shot Chart Detail,21400010,112,203112,Quincy Acy,1610612752,New York Knicks,1,0,...,Center(C),16-24 ft.,16,-23,159,1,0,20141029,NYK,CHI
2,2,Shot Chart Detail,21400010,396,203112,Quincy Acy,1610612752,New York Knicks,4,10,...,Left Side Center(LC),16-24 ft.,17,-71,162,1,1,20141029,NYK,CHI
3,3,Shot Chart Detail,21400010,436,203112,Quincy Acy,1610612752,New York Knicks,4,6,...,Left Side(L),8-16 ft.,11,-97,60,1,0,20141029,NYK,CHI
4,4,Shot Chart Detail,21400010,454,203112,Quincy Acy,1610612752,New York Knicks,4,5,...,Left Side Center(LC),16-24 ft.,19,-144,129,1,0,20141029,NYK,CHI


In [63]:
shots_1415.drop(columns = 'Unnamed: 0')

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,21400010,107,203112,Quincy Acy,1610612752,New York Knicks,1,1,22,...,Right Side(R),16-24 ft.,18,185,15,1,1,20141029,NYK,CHI
1,Shot Chart Detail,21400010,112,203112,Quincy Acy,1610612752,New York Knicks,1,0,31,...,Center(C),16-24 ft.,16,-23,159,1,0,20141029,NYK,CHI
2,Shot Chart Detail,21400010,396,203112,Quincy Acy,1610612752,New York Knicks,4,10,26,...,Left Side Center(LC),16-24 ft.,17,-71,162,1,1,20141029,NYK,CHI
3,Shot Chart Detail,21400010,436,203112,Quincy Acy,1610612752,New York Knicks,4,6,32,...,Left Side(L),8-16 ft.,11,-97,60,1,0,20141029,NYK,CHI
4,Shot Chart Detail,21400010,454,203112,Quincy Acy,1610612752,New York Knicks,4,5,55,...,Left Side Center(LC),16-24 ft.,19,-144,129,1,0,20141029,NYK,CHI
5,Shot Chart Detail,21400010,463,203112,Quincy Acy,1610612752,New York Knicks,4,5,4,...,Center(C),Less Than 8 ft.,1,4,14,1,1,20141029,NYK,CHI
6,Shot Chart Detail,21400018,42,203112,Quincy Acy,1610612752,New York Knicks,1,6,5,...,Center(C),16-24 ft.,20,58,200,1,0,20141030,CLE,NYK
7,Shot Chart Detail,21400018,55,203112,Quincy Acy,1610612752,New York Knicks,1,4,37,...,Center(C),Less Than 8 ft.,0,0,7,1,1,20141030,CLE,NYK
8,Shot Chart Detail,21400018,173,203112,Quincy Acy,1610612752,New York Knicks,2,6,49,...,Center(C),Less Than 8 ft.,0,0,4,1,1,20141030,CLE,NYK
9,Shot Chart Detail,21400018,253,203112,Quincy Acy,1610612752,New York Knicks,3,11,5,...,Right Side(R),8-16 ft.,13,108,89,1,0,20141030,CLE,NYK


In [67]:
shots_1415.shape

(198165, 25)

In [64]:
all_shots_1516 = get_all_players_shot_data(player_ids, '2015-16')
shots_1516_df, lg_avgs_1516 = convert_dict_to_df(all_shots_1516)

shots_1516_df.to_csv('data/shots_1515.csv')

Runtime: 89.80601119995117 seconds. 50 players completed
Runtime: 178.5865683555603 seconds. 100 players completed
Runtime: 265.4269881248474 seconds. 150 players completed
Runtime: 353.4865572452545 seconds. 200 players completed
Runtime: 441.85187125205994 seconds. 250 players completed
Runtime: 527.3546442985535 seconds. 300 players completed
Runtime: 612.0405220985413 seconds. 350 players completed
Runtime: 696.8318672180176 seconds. 400 players completed
Runtime: 781.6106140613556 seconds. 450 players completed
Runtime: 877.1560802459717 seconds. 500 players completed
Runtime: 969.2065002918243 seconds. 550 players completed
Runtime: 1060.5570213794708 seconds. 600 players completed
Runtime: 1153.3278074264526 seconds. 650 players completed
Runtime: 1242.5283992290497 seconds. 700 players completed
Total Runtime: 1.7739410400390625 seconds.


In [65]:
all_shots_1617 = get_all_players_shot_data(player_ids, '2016-17')
shots_1617_df, lg_avgs_1617 = convert_dict_to_df(all_shots_1617)

shots_1617_df.to_csv('data/shots_1617.csv')

Runtime: 88.4657130241394 seconds. 50 players completed
Runtime: 185.22618222236633 seconds. 100 players completed
Runtime: 276.0623321533203 seconds. 150 players completed
Runtime: 366.89074301719666 seconds. 200 players completed
Runtime: 1088.024829864502 seconds. 250 players completed
Runtime: 1182.0320520401 seconds. 300 players completed
Runtime: 1273.1256380081177 seconds. 350 players completed
Runtime: 1709.764703989029 seconds. 400 players completed
Runtime: 1819.5310490131378 seconds. 450 players completed
Runtime: 1925.21572804451 seconds. 500 players completed
Runtime: 2182.9147040843964 seconds. 550 players completed
Runtime: 2273.4975979328156 seconds. 600 players completed
Runtime: 2372.150113105774 seconds. 650 players completed
Runtime: 2466.349255800247 seconds. 700 players completed
Total Runtime: 2.1243081092834473 seconds.


In [66]:
all_shots_1718 = get_all_players_shot_data(player_ids, '2017-18')
shots_1718_df, lg_avgs_1718 = convert_dict_to_df(all_shots_1718)

shots_1718_df.to_csv('data/shots_1718.csv')

Runtime: 85.98297119140625 seconds. 50 players completed
Runtime: 170.821387052536 seconds. 100 players completed
Runtime: 257.7611529827118 seconds. 150 players completed
Runtime: 344.3887982368469 seconds. 200 players completed
Runtime: 535.7709269523621 seconds. 250 players completed
Runtime: 623.221559047699 seconds. 300 players completed
Runtime: 712.1062681674957 seconds. 350 players completed
Runtime: 800.8886630535126 seconds. 400 players completed
Runtime: 901.6472570896149 seconds. 450 players completed
Runtime: 990.7329950332642 seconds. 500 players completed
Runtime: 1087.5008292198181 seconds. 550 players completed
Runtime: 1221.957328081131 seconds. 600 players completed
Runtime: 1329.993782043457 seconds. 650 players completed
Runtime: 1417.8486320972443 seconds. 700 players completed
Total Runtime: 1.7772679328918457 seconds.


In [68]:
all_shots_1314 = get_all_players_shot_data(player_ids, '2013-14')
shots_1314_df, lg_avgs_1314 = convert_dict_to_df(all_shots_1314)

shots_1314_df.to_csv('data/shots_1314.csv')

Runtime: 85.97768306732178 seconds. 50 players completed
Runtime: 170.45652103424072 seconds. 100 players completed
Runtime: 255.74277925491333 seconds. 150 players completed
Runtime: 345.3604521751404 seconds. 200 players completed
Runtime: 438.643559217453 seconds. 250 players completed
Runtime: 527.2146461009979 seconds. 300 players completed
Runtime: 1387.3804681301117 seconds. 350 players completed
Runtime: 1477.4999990463257 seconds. 400 players completed
Runtime: 1566.1757082939148 seconds. 450 players completed
Runtime: 1654.062170267105 seconds. 500 players completed
Runtime: 1740.061388015747 seconds. 550 players completed
Runtime: 1829.5794742107391 seconds. 600 players completed
Runtime: 1923.8664100170135 seconds. 650 players completed
Runtime: 2011.4249789714813 seconds. 700 players completed
Total Runtime: 1.486617088317871 seconds.


In [70]:
all_shots_1213 = get_all_players_shot_data(player_ids, '2012-13')
shots_1213_df, lg_avgs_1213 = convert_dict_to_df(all_shots_1213)

shots_1213_df.to_csv('data/shots_1213.csv')

Runtime: 66.72446274757385 seconds. 50 players completed
Runtime: 122.68030095100403 seconds. 100 players completed
Runtime: 178.60781693458557 seconds. 150 players completed
Runtime: 234.58194875717163 seconds. 200 players completed
Runtime: 290.87318778038025 seconds. 250 players completed
Runtime: 348.1546697616577 seconds. 300 players completed
Runtime: 404.63295102119446 seconds. 350 players completed
Runtime: 470.81017088890076 seconds. 400 players completed
Runtime: 798.4196360111237 seconds. 450 players completed
Runtime: 880.4472026824951 seconds. 500 players completed
Runtime: 964.0385658740997 seconds. 550 players completed
Runtime: 1048.0967857837677 seconds. 600 players completed
Runtime: 1132.3736398220062 seconds. 650 players completed
Runtime: 1216.0260260105133 seconds. 700 players completed
Total Runtime: 1.4525198936462402 seconds.


In [72]:
shots_1213 = pd.read_csv('data/shots_1213.csv')
shots_1213.shape

(171003, 25)

In [75]:
print(shots_1213.describe())

          Unnamed: 0       GAME_ID  GAME_EVENT_ID      PLAYER_ID  \
count  171003.000000  1.710030e+05  171003.000000  171003.000000   
mean    85501.000000  2.120062e+07     246.308989  143605.418998   
std     49364.458378  3.544626e+02     154.105838   86005.628267   
min         0.000000  2.120000e+07       2.000000     708.000000   
25%     42750.500000  2.120031e+07     110.000000  101106.000000   
50%     85501.000000  2.120062e+07     244.000000  201566.000000   
75%    128251.500000  2.120092e+07     372.000000  202328.000000   
max    171002.000000  2.120123e+07     763.000000  203382.000000   

            TEAM_ID         PERIOD  MINUTES_REMAINING  SECONDS_REMAINING  \
count  1.710030e+05  171003.000000      171003.000000      171003.000000   
mean   1.610613e+09       2.471775           5.308930          28.687029   
std    8.644529e+00       1.139770           3.449656          17.472625   
min    1.610613e+09       1.000000           0.000000           0.000000   
25%    

In [76]:
shots_1213.columns

Index(['Unnamed: 0', 'GRID_TYPE', 'GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID',
       'PLAYER_NAME', 'TEAM_ID', 'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING',
       'SECONDS_REMAINING', 'EVENT_TYPE', 'ACTION_TYPE', 'SHOT_TYPE',
       'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE',
       'LOC_X', 'LOC_Y', 'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG', 'GAME_DATE',
       'HTM', 'VTM'],
      dtype='object')

In [80]:
all_shots_1112 = get_all_players_shot_data(player_ids, '2011-12')
shots_1112_df, lg_avgs_1112 = convert_dict_to_df(all_shots_1112)

shots_1112_df.to_csv('data/shots_1112.csv')

Runtime: 71.05837798118591 seconds. 50 players completed
Runtime: 244.42249488830566 seconds. 100 players completed
Runtime: 328.90730118751526 seconds. 150 players completed
Runtime: 416.3233232498169 seconds. 200 players completed
Runtime: 501.345498085022 seconds. 250 players completed
Runtime: 585.4117259979248 seconds. 300 players completed
Runtime: 669.3578989505768 seconds. 350 players completed
Runtime: 757.4456830024719 seconds. 400 players completed
Runtime: 845.9135019779205 seconds. 450 players completed
Runtime: 934.5969190597534 seconds. 500 players completed
Runtime: 1023.4780158996582 seconds. 550 players completed
Runtime: 1108.8588740825653 seconds. 600 players completed
Runtime: 1199.5030829906464 seconds. 650 players completed
Runtime: 1285.8225700855255 seconds. 700 players completed
Total Runtime: 1.2211229801177979 seconds.


In [81]:
all_shots_1011 = get_all_players_shot_data(player_ids, '2010-11')
shots_1011_df, lg_avgs_1011 = convert_dict_to_df(all_shots_1011)

shots_1011_df.to_csv('data/shots_1011.csv')

Runtime: 86.6445460319519 seconds. 50 players completed
Runtime: 169.90437698364258 seconds. 100 players completed
Runtime: 264.1239140033722 seconds. 150 players completed
Runtime: 349.62002301216125 seconds. 200 players completed
Runtime: 436.56049609184265 seconds. 250 players completed
Runtime: 520.3242678642273 seconds. 300 players completed
Runtime: 605.3194899559021 seconds. 350 players completed
Runtime: 690.8187491893768 seconds. 400 players completed
Runtime: 903.1392970085144 seconds. 450 players completed
Runtime: 1041.9745450019836 seconds. 500 players completed
Runtime: 1128.0681459903717 seconds. 550 players completed
Runtime: 1214.9839038848877 seconds. 600 players completed
Runtime: 1299.7993259429932 seconds. 650 players completed
Runtime: 1388.9560339450836 seconds. 700 players completed
Total Runtime: 1.166957139968872 seconds.


In [82]:
all_shots_0910 = get_all_players_shot_data(player_ids, '2009-10')
shots_0910_df, lg_avgs_0910 = convert_dict_to_df(all_shots_0910)

shots_0910_df.to_csv('data/shots_0910.csv')

Runtime: 87.83940720558167 seconds. 50 players completed
Runtime: 174.06075310707092 seconds. 100 players completed
Runtime: 260.2879810333252 seconds. 150 players completed
Runtime: 346.2997691631317 seconds. 200 players completed
Runtime: 432.37960290908813 seconds. 250 players completed
Runtime: 516.9954109191895 seconds. 300 players completed
Runtime: 603.9341771602631 seconds. 350 players completed
Runtime: 688.7237229347229 seconds. 400 players completed
Runtime: 773.9198362827301 seconds. 450 players completed
Runtime: 857.0653991699219 seconds. 500 players completed
Runtime: 943.4973990917206 seconds. 550 players completed
Runtime: 1030.8029980659485 seconds. 600 players completed
Runtime: 1115.540428161621 seconds. 650 players completed
Runtime: 1202.2598631381989 seconds. 700 players completed
Total Runtime: 1.2149858474731445 seconds.


In [ ]:
import pandas as pd
import requests
import math
import time
import random


pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_row', 100)
pd.set_option('display.max_columns', 50)


def GeneratePlayerInfo():
    
    allPlayerIds = pd.read_csv('shot_logs.csv')
    
    allPlayerIds.rename(columns= {'player_id':'PLAYER_ID'}, inplace= True)
    
    allPlayerIds = allPlayerIds.drop_duplicates('PLAYER_ID')
    
    justIds = allPlayerIds['PLAYER_ID']
    
    headers_list = [
        "PERSON_ID",
        "FIRST_NAME",
        "LAST_NAME",
        "DISPLAY_FIRST_LAST",
        "DISPLAY_LAST_COMMA_FIRST",
        "DISPLAY_FI_LAST",
        "BIRTHDATE",
        "SCHOOL",
        "COUNTRY",
        "LAST_AFFILIATION",
        "HEIGHT",
        "WEIGHT",
        "SEASON_EXP",
        "JERSEY",
        "POSITION",
        "ROSTERSTATUS",
        "TEAM_ID",
        "TEAM_NAME",
        "TEAM_ABBREVIATION",
        "TEAM_CODE",
        "TEAM_CITY",
        "PLAYERCODE",
        "FROM_YEAR",
        "TO_YEAR",
        "DLEAGUE_FLAG",
        "NBA_FLAG",
        "GAMES_PLAYED_FLAG",
        "DRAFT_YEAR",
        "DRAFT_ROUND",
        "DRAFT_NUMBER"
        ]
    
    player_info = []
    
    for id in justIds:
        if (id == 0) | (id is None) | (math.isnan(id)):
            print('skipping')
            print(id)
            continue
            
        print(id)
        
        playerid = int(id)
        # Testing
        # playerid = 203148
        
        headers = {'User-Agent': 'PostmanRuntime/7.4.0'}
        url = "https://stats.nba.com/stats/commonplayerinfo?LeagueID=&PlayerID=" + str(playerid)
        
        response = requests.get(url, headers=headers)
        playerData = response.json()
        
        playerD = playerData['resultSets'][0] if 'resultSets' in playerData and len(playerData['resultSets']) > 0 else {}
        rs = playerD['rowSet'] if 'rowSet' in playerD else {}
        info = rs[0] if len(rs) >= 0 else []
        
        if(len(info) != 30): # Length of info should always be 29 to match headers
            print("Length doesn't match or error parsing for info")
            continue
            
        player_info.append(info)
        time.sleep(random.uniform(.1, .2))
        
    df = pd.DataFrame(player_info, columns=headers_list)
    df.to_csv('test1.csv')
    
if __name__ == '__main__':
    GeneratePlayerInfo()
    # GeneratePlayerInfo()

In [24]:
import pandas as pd
import requests
import math
import time
import random


pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_row', 100)
pd.set_option('display.max_columns', 50)


def GenerateDefPlayerInfo():
    
    allPlayerIds = pd.read_csv('shots_df_1415.csv')
    
    DefPlayerIds = allPlayerIds.drop_duplicates('CLOSEST_DEFENDER_PLAYER_ID')
    OffPlayerIds = allPlayerIds.drop_duplicates('PLAYER_ID')   

    justDefIds = DefPlayerIds['CLOSEST_DEFENDER_PLAYER_ID'].tolist()
    justOffIds = OffPlayerIds['PLAYER_ID'].tolist()
    
    c = [i for i in justDefIds if i not in justOffIds]
    diffIdList = c.copy()
    diffIdList.remove(530027)
    diffIdList.remove(203296)
    
    headers_list = [
        "PERSON_ID",
        "FIRST_NAME",
        "LAST_NAME",
        "DISPLAY_FIRST_LAST",
        "DISPLAY_LAST_COMMA_FIRST",
        "DISPLAY_FI_LAST",
        "BIRTHDATE",
        "SCHOOL",
        "COUNTRY",
        "LAST_AFFILIATION",
        "HEIGHT",
        "WEIGHT",
        "SEASON_EXP",
        "JERSEY",
        "POSITION",
        "ROSTERSTATUS",
        "TEAM_ID",
        "TEAM_NAME",
        "TEAM_ABBREVIATION",
        "TEAM_CODE",
        "TEAM_CITY",
        "PLAYERCODE",
        "FROM_YEAR",
        "TO_YEAR",
        "DLEAGUE_FLAG",
        "NBA_FLAG",
        "GAMES_PLAYED_FLAG",
        "DRAFT_YEAR",
        "DRAFT_ROUND",
        "DRAFT_NUMBER"
        ]
    
    player_info = []
    
    for id in diffIdList:
        if (id == 0) | (id is None) | (math.isnan(id)):
            print('skipping')
            print(id)
            continue
            
        print(id)
        
        playerid = int(id)
        # Testing
        # playerid = 203148
        
        headers = {'User-Agent': 'PostmanRuntime/7.4.0'}
        url = "https://stats.nba.com/stats/commonplayerinfo?LeagueID=&PlayerID=" + str(playerid)
        
        response = requests.get(url, headers=headers)
        playerData = response.json()
        
        playerD = playerData['resultSets'][0] if 'resultSets' in playerData and len(playerData['resultSets']) > 0 else {}
        rs = playerD['rowSet'] if 'rowSet' in playerD else {}
        info = rs[0] if len(rs) >= 0 else []
        
        if(len(info) != 30): # Length of info should always be 29 to match headers
            print("Length doesn't match or error parsing for info")
            continue
            
        player_info.append(info)
        time.sleep(random.uniform(1, 2))
        
    df = pd.DataFrame(player_info, columns=headers_list)
    df.to_csv('test1.csv')
    
if __name__ == '__main__':
    GenerateDefPlayerInfo()
    # GeneratePlayerInfo()

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (70,71,72) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


203482
2746
201943
203484
202328
203460
203103
201152
101112
203901
202693
203101
203110
201149
2223
2405
200794
2760
201959
2399
2200
202703
201601
2617
202708
203083
201581
202702
2561
201588
201574
202696
203124
202699
201596
203096
202700
2403
203078
201954
203957
202683
202344
201143
201942
101161
200755
2440
101108
202349
201160
203089
201162
201144
2216
201150
201568
201951
202340
2570
2555
2742
201937
203461
2584
101106
2581
203086
200761
2365
203507
201586
203496
203156
203909
203499
203118
202684
201567
201565
2736
203521
202681
2034
201973
202323
203935
203492
202355
203092
101181
201175
202706
203500
203112
201609
201577
2547
203953
202498
203457
1890
202710
203138
2544
201145
203524
200797
200752
201933
201599
202685
200811
202066
201564
202326
2730
2772
1891
202694
202339
203458
202691
203084
2738
200782
203077
101107
202362
2754
201569
201950
203085
202337
201980
202695
1938
201939
200757
203145
204028
201976
101128
202718
201148
201163
201167
201580
203952
2590
203082
2

KeyError: 0

In [ ]:
# every time fails after 530027... time to remove the following id